## AMAL - TP 6 - 正则化与实验方案

在本次实验课中，我们将关注那些有助于提升泛化能力的机制。

- L1 和 L2 正则化
- Dropout
- BatchNorm（批归一化）
- LayerNorm（层归一化）

对于 Dropout 和 BatchNorm，两者的归一化在推理和训练时的行为有所不同。在 PyTorch 中，为指示当前模式，可以调用模块的 `eval()` 和 `train()` 方法。

本次实验课也将介绍两个对实验非常有用的模块：`lightning`（可以大幅简化实验代码）和 `optuna`（可以优化超参数搜索）。

### 1 数据和模型的准备  

您将使用 MNIST 数据集。

- #### 重要

在本次实验中，您只使用训练数据的 5%（否则正则化技术将不会有任何效果）。  

```python
from datamaestro import prepare_dataset
ds = prepare_dataset("com.lecun.mnist")

# 不要忘记进行下采样！
train_img, train_labels = ds.train.images.data(), ds.train.labels.data()
test_img, test_labels = ds.test.images.data(), ds.test.labels.data()
```

您将使用一个由 3 个线性层组成的网络，每层有 100 个输出，接着是一个用于分类的线性层（10 个类别，表示数字 0 到 9）。您将使用交叉熵损失函数、批量大小为 300，并进行 1000 次迭代（epochs）。

- #### 问题 1

为了观察正则化的效果，需要使用 `torch.utils.tensorboard` 记录以下内容：

- 损失值（训练、验证和测试集的损失）。
- 每个线性层的权重（直方图）。
- 每个线性层输入的梯度；要记录某个变量的梯度，可以使用 `store_grad` 函数或张量的 `retain_grad` 方法。
- 输出的熵（直方图）；快速计算一个随机模型的熵作为对比。

注意：避免在每次迭代中记录直方图；在训练过程中最多记录约 20 次，以节省存储空间和计算时间。

### 2 模型正则化  
#### $L1$ 和 $L2$ 正则化

L1 或 L2 正则化对应于对模型参数设置一个先验分布。通过最大后验概率准则（MAP），我们可以使用拉普拉斯先验（用于 L1）或高斯先验（用于 L2）。实际上，我们试图最大化：

$$
p(\theta | \text{données}) = \frac{p(\text{données} | \theta) p(\theta)}{p(\text{données})}
$$

翻译如下：

这等同于最小化

$$
\log p(\theta | \text{données}) = \text{constante} - \log p(\text{données} | \theta) - \log p(\theta)
$$

对于一个 Laplace 先验 $p(\theta)$，我们有：
$$
\log p(\theta) = \text{constante} + \underbrace{\frac{1}{b}}_{\lambda _1}  \|\theta\|_1
$$

对于一个 Gaussian 先验 $p(\theta)$，我们有：
$$
\log p(\theta) = \text{constante} + \underbrace{\frac{1}{2\sigma^2}}_{\lambda _2}  \|\theta\|_2^2
$$

因此，$L1/L2$ 正则化有一个超参数需要调整。在实际应用中，这个超参数越大，参数偏离 $0$ 的程度就越小。请观察其效果。

#### Dropout  
Dropout 的理念与模型平均类似：通过独立训练 k 个模型，我们可以降低模型的方差。训练 k 个模型会带来不小的开销，而 Dropout 的优点在于减少了内存/计算时间的复杂性。其思路是在 Dropout 层中以概率 p（一个超参数）将每个输出置为零。

- #### 重要

在推理过程中，这个概率 p 必须被忽略（并且需要调整值，因为所有输入都存在，需要乘以 $ (1 - p)^{-1} $）。为此，需要指定模型是处于训练状态（使用 `model.train()`）还是在推理中使用（使用 `model.eval()`）。

#### 批量归一化（BatchNorm）  
我们知道，中心化和标准化的数据可以更快、更稳定地训练模型；虽然可以确保输入数据是中心化和标准化的，但对于神经网络的内部层，这就更为复杂。批量归一化技术的目的是通过使用滑动平均/方差（在推理时）和批次统计（在训练时）来添加一层，以中心化/标准化数据。  

使用 `BatchNorm1d` 类。

- #### 重要
同样，这里也有一个区分训练和推理的模式：在推理模式下，均值和方差将不再更新。

#### 层归一化（LayerNorm）
层归一化（LayerNorm）是一种替代批归一化（BatchNorm）的方法，主要用于当我们没有足够的数据来估计方差和均值时（例如，在强化学习中）。它的工作原理是对每个个体进行数据归一化（而不是像批归一化那样按特征进行归一化）。

使用 `LayerNorm` 类。

- #### 问题 2
更改您的模型以使用上述提到的不同正则化方法。某些正则化方法可以结合使用（例如，L1/L2 与 Dropout、BatchNorm 或 LayerNorm）。

### 3 数据增强（可选）

另一种避免过拟合的技术是人工创建更多的数据。您可以使用 torchvision 库中提供的函数。

- #### 问题 3

为此，可以进行一些不改变预期类别的变换：
- 添加高斯噪声
- 对于图像，进行一些无损的几何变换（旋转、缩放等）

### 4 Lightning

Lightning 允许简化实验部分的代码并自动化一些冗余的操作。它特别能够：
- 简化训练循环、检查点管理和日志记录
- 对硬件保持不变（支持 GPU、多 GPU 和 CPU）
- 灵活地切换不同的训练/评估/测试模式

通常需要两个类：一个实现模型行为的类，继承自 `LightningModule`，以及一个用于数据加载的类，继承自 `LightningDataModule`。

一个经典的模型代码示例在 `lightningexemple.py` 文件中。可以找到常规的 `forward` 方法，该方法用于计算模型的输出。`configure_optimizers` 方法用于指定所使用的优化器。然后，只需编写 `training_step`、`validation_step` 和 `test_step` 方法，这些方法分别执行一次学习、验证和测试迭代：至少需要在批次上应用模型并计算损失。注意，代码中没有对 `backward`、优化器的显式调用，也没有将模型和数据转移到 GPU 或 CPU 的操作：所有这些都被封装在内。

接下来，需要创建一个 `Trainer` 并调用该训练器的 `fit` 方法来训练模型：所有调用将自动执行。

`LitMnistData` 类给出了数据集管理的示例：需要有一个 `prepare_dataset` 和 `setup` 方法来准备数据（下载、预处理），以及 `train_dataloader`、`val_dataloader` 和 `test_dataloader` 方法，分别返回训练、验证和测试数据。

许多钩子可用于灵活地使用模块，例如 `training_epoch_end`，它允许在训练周期结束时接管控制，以便计算要记录的量（如直方图、图像或其他）。请查看文档以获取更多信息。

### 5 超参数搜索

您将尝试使用剩余 95% 的训练数据集进行最佳超参数的搜索。您可以尝试结合不同形式的正则化。目前有许多库可供使用（例如，仍在开发中的 nevergrad），但我们推荐使用 optuna，因为它可以轻松处理超参数之间的依赖关系。

- #### 问题 4
使用 optuna 库实现超参数搜索，以优化超参数。